# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [16]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [17]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Чистые понятные данные, думаю более глубокий их анализ в данном контексте не требуется

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** ...

**Обоснование:** ...

**Ответ:** не измениться

**Обоснование:** Для обоснования данного ответа используем следующие свойства матричного умножения
*  $ AB \not\equiv BA $ 
*  $(AB)^{-1} = B^{-1}A^{-1} $
*  $(AB)^{T} = B^{T}A^{T} $ 
* $AA^{-1} = E $ 
* $АЕ = ЕА = А $ 

Исходную формулу:
$$
w = (X^T X)^{-1} X^T y
$$

Домнажаем на матрицу P:
$$
w1 = ((XP)^T XP)^{-1} (XP)^T y
$$

Используя $(AB)^{-1} = B^{-1}A^{-1} $, преобразуем формулу:
$$
w1 = (P^TX^T XP)^{-1} P^TX^T y
$$

Раскроем скобки используя $(AB)^{-1} = B^{-1}A^{-1} $ :
$$
w1 = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^T y
$$

Сократим  $P^T(P^T)^{-1}= E$, отметим что P квадратная матрица:
$$
w1 = P^{-1}(X^TX)^{-1}X^T y
$$

По имеющейся формуле $ w = (X^T X)^{-1} X^T y $ получаем:
$$
w1 = P^{-1}w
$$

Так же имеется $a = Xw$:
$$
a1 = XPw1
$$

Подставим $w1 = P^{-1}w$  и сократим:
$$
a1 = XPP^{-1}w = Xw = a
$$

Из полученных результатов можно сделать вывод что качество модели R2 не измениться


## Алгоритм преобразования

**Алгоритм**

Создадим квадратную матрицу P и проверим ее на обратимость.

Исходую матрицу признаков X  домножим на квадратную обратимую матрицу P (мы доказали, что так можно и результат с качеством не изменятся), так же при необходимости можно домножить на обратную P матрицу и востановить зашифрованные данные.

## Проверка алгоритма

Для проверки алгоритма разобьем данные

In [19]:
features = data.drop(columns='Страховые выплаты', axis=1)
target = data['Страховые выплаты']
features_train, features_test, target_train, target_test = train_test_split(
    features,    target,     random_state=12345,shuffle=True,   test_size=0.25)

Создадим шифрующую матрицу и проверим ее обратимость

In [20]:
P = np.random.rand(4,4)

np.linalg.inv(P)

array([[ 0.91166429,  1.24453832,  1.77085455, -2.45803073],
       [-2.65871066, -0.18596162, -1.24511119,  2.37405702],
       [ 1.99222361,  1.34839289, -0.20909839, -1.36652949],
       [ 0.07748192, -1.75853636,  0.35077861,  1.59683567]])

Посчитаем R2 без шифрования.

In [21]:
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_test)
    r2_score(target_test, predictions)

0.43522757127026546

Сделаем все тоже самое только с шифрованием

In [22]:
    features_train = features_train.dot(P)
features_test = features_test.dot(P)
    
    model1 = LinearRegression()
    model1.fit(features_train, target_train)
    predictions = model1.predict(features_test)
    r2_score(target_test, predictions)

0.43522757127010403

**Вывод:**  метрика r2 не изменилась, следовательно данный метод шифрования не сказывается на качестве предсказания, а мы можем зашифровать данные клиента.